# Exercises

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

### 1.  A bank found that the average number of cars waiting during the noon hour at a drive-up window follows a Poisson distribution with a mean of 2 cars. Make a chart of this distribution and answer these questions concerning the probability of cars waiting at the drive-up window.

- a. What is the probability that no cars drive up in the noon hour?
- b. What is the probability that 3 or more cars come through the drive through?
- c. How likely is it that the drive through gets at least 1 car?

In [3]:
mean = 2
x = np.arange(0,1)
y = stats.poisson(λ).
plt.bar(x,y)
plt.title("Likelyhood of N number of cars appearing at this Bank's drive-up window at Noon")
plt.show()

NameError: name 'y' is not defined

# a.

# b.

# c.